In [1]:
from importlib import reload
import numpy as np
import time
from time import sleep 
import pyautogui
import pydirectinput
import pandas as pd

#import main
#reload(main)
import fct
reload(fct)


<module 'fct' from 'c:\\Users\\polo\\Pictures\\Cours Info\\Bot trading\\Undecember\\fct.py'>

# Test déplacement souris

In [ ]:
pip install pickle


In [321]:


wave

,Unnamed: 0.1,Unnamed: 0,farm_act,go_blacksmith,go_storage,go_tower,Pirate_1,Pirate_boss
0,0,0,"[1550, 260]","[1260, 90]","[1605, 60]","[2050, 130]","[200, 1300]","[700, 700]"
1,1,1,"[2000, 1150]","[1260, 90]","[1470, 577]","[2050, 130]","[600, 1300]","[700, 700]"
2,2,2,"[2200, 1300]","[1050, 650]",NaN,"[1700, 130]","[900, 1200]","[700, 700]"
3,3,3,"[2200, 150]",NaN,NaN,"[1900, 130]","[400, 260]","[700, 700]"
4,4,4,"[2200, 150]",NaN,NaN,"[1785, 645]","[460, 700]","[700, 700]"
5,5,5,"[2200, 150]",NaN,NaN,NaN,"[400, 1200]","[1000, 450]"
6,6,6,"[2200, 150]",NaN,NaN,NaN,"[1300, 1000]",NaN
7,7,7,"[1640, 120]",NaN,NaN,NaN,"[1300, 1000]",NaN
8,8,8,"[1640, 120]",NaN,NaN,NaN,"[1300, 1000]",NaN
9,9,9,"[1640, 120]",NaN,NaN,NaN,"[1300, 1000]",NaN


In [ ]:
path_img='screen/launch/take_waypoint.png'
pyautogui.locateOnScreen(path_img, grayscale=True, confidence=0.80)

In [5]:
import re
import math

a=wave['Pirate_1']

fct.move_trajet(a)

KeyboardInterrupt: 

In [239]:
cpt=0

In [213]:
cpt-=1
cpt

5

In [241]:
wave["Pirate_boss"]=''

In [249]:
wave["Pirate_boss"][cpt]= [ 1000, 1000  ]

pyautogui.leftClick(wave.loc[cpt,"Pirate_boss"])
cpt+=1

C:\Users\polo\AppData\Local\Temp\ipykernel_11300\1281230767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave["Pirate_boss"][cpt]= [ 1000, 1000  ]


In [2]:
wave=pd.read_csv('wave.csv')
wave


,farm_act,go_blacksmith,go_storage,go_tower,Pirate_1,Pirate_boss
0,"[1550, 260]","[1260, 90]","[1605, 60]","[2050, 130]","917,1144","[700, 700]"
1,"[2000, 1150]","[1260, 90]","[1470, 577]","[2050, 130]","917,1143","[700, 700]"
2,"[2200, 1300]","[1050, 650]",NaN,"[1700, 130]","809,879","[700, 700]"
3,"[2200, 150]",NaN,NaN,"[1900, 130]","812,760","[700, 700]"
4,"[2200, 150]",NaN,NaN,"[1785, 645]","812,745","[700, 700]"
...,...,...,...,...,...,...
180,NaN,NaN,NaN,NaN,"864,1090",NaN
181,NaN,NaN,NaN,NaN,"705,876",NaN
182,NaN,NaN,NaN,NaN,"798,1009",NaN
183,NaN,NaN,NaN,NaN,"798,994",NaN


In [320]:
wave.to_csv('wave.csv')

In [318]:
find is not None

False

In [317]:
path=f'screen/launch/{m}.png'
find=pyautogui.locateOnScreen(path, grayscale=False, confidence=0.95)
if find is not None:
        find = pyautogui.center(find)
        pyautogui.leftClick(find,duration=0.3)
        time.sleep(np.random.uniform(0.4, 0.8))

In [319]:
import fct
reload(fct)
list_map=["Pirate" ]

def test_in_game():
    path=f'screen/launch/test_in_game.png'
    find=pyautogui.locateOnScreen(path, grayscale=False, confidence=0.95)
    if find is None:
        if fct.launch() is None:
            1

class Perso():
    def __init__(self):
        test_in_game()
            
        self.actif=False
    
    
    def filtre_map(self):
        click_icone('screen/launch/start_filtre.png')
        click_icone('screen/launch/filtre_apply.png',1)
        click_icone('screen/launch/select_all.png',2)
        click_icone('screen/launch/select_all.png',2)
        
        click_icone('screen/launch/Ganida.png',5,0.3,False)
        
        click_icone('screen/launch/tier10.png',1)
        click_icone('screen/launch/ok.png',10,0.3,False,0.7)
        self.actif=True
    
    def launch_map(self):
        for m in list_map:
            path=f'screen/launch/{m}.png'
            find = pyautogui.locateOnScreen(path, grayscale=False, confidence=0.95)
            if find is not None:
                find = pyautogui.center(find)
                pyautogui.rightClick(find,duration=0.3)
                time.sleep(np.random.uniform(0.4, 0.8))
                continue
            
            
        click_icone('screen/launch/activate.png',3,0.3,False,0.8)
        time.sleep(np.random.uniform(3.5, 5.8))
    
        pyautogui.leftClick(830,1200,duration=0.3)
        fct.keyboard('f')
    
    
perso=Perso()
#perso.filtre_map() 
#fct.go_tower(wave)  
perso.launch_map()

    
#fct.go_blacksmith(wave)      

#fct.go_tower(wave)    
    
#fct.go_storage(wave)




In [260]:
import fct
reload(fct)

fct.go_blacksmith(wave)
fct.go_storage(wave)

Looking For screen/launch/arme.png
Looking For screen/launch/normal.png
Looking For screen/launch/magic.png
Looking For screen/launch/rare.png
Looking For screen/launch/normal.png
Looking For screen/launch/magic.png
Looking For screen/launch/rare.png


In [192]:
def click_icone(path,boucle=10,wait=0.3,gris=True,confidence=0.95):
    while boucle > 0:
        time.sleep(np.random.uniform(wait, wait*3)) 
        find = pyautogui.locateOnScreen(path, grayscale=gris, confidence=confidence)
        if find is not None:
            find = pyautogui.center(find)
            pyautogui.leftClick(find,duration=0.3)
            break
        else:
            print(f'Looking For {path}')
        boucle -=1


In [ ]:
import pyautogui
import pydirectinput
sleep(1)
pydirectinput.moveTo(200, 150) # Move the mouse to the x, y coordinates 100, 150.
pydirectinput.click() # Click the mouse at its current location.
pydirectinput.click(200, 220) # Click the mouse at the x, y coordinates 200, 220.
sleep(1)
pydirectinput.move(None, 100)  # Move mouse 10 pixels down, that is, move the mouse relative to its current position.
pydirectinput.doubleClick() # Double click the mouse at the
pydirectinput.press('esc') # Simulate pressing the Escape key.
pydirectinput.keyDown('shift')
pydirectinput.keyUp('shift')
pydirectinput.moveTo(1200, 1150) # Move the mouse to the x, y coordinates 100, 150.
pydirectinput.doubleClick() 
pydirectinput.click()

In [ ]:
pydirectinput.moveTo(1200, 650)#raccourci undecember
pydirectinput.doubleClick() 

pydirectinput.moveTo(2300, 570)#start
pydirectinput.click() 

pydirectinput.moveTo(2300, 1312)#start 2
pydirectinput.click() 